# 연습문제
1. 네이버 오픈 API 를 활용하여 네이버지식인 "전주여행" 과 "경주여행"을 검색
   1. pip install python-dotenv
2. 명사를 추출
3. 빈도분석 (DataFrame)
4. 빈도시각화 (워드클라우드, nltk.Text)
5. Word2Vec




# 1. 네이버 open API 를 활용하여 검색 추출

- 검색어, no, title, link, description + title + '' + description

In [ ]:
import os
import sys
import urllib.request
import json
import re
from dotenv import load_dotenv
from konlpy.tag import Okt
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import Text
from gensim.models import Word2Vec
import platform

load_dotenv()
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

encText = urllib.parse.quote("전주여행")
url = f"https://openapi.naver.com/v1/search/kin.json?query={encText}&display=99" 
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
else:
    print("Error Code:" + rescode)
    sys.exit()

data = json.loads(response_body.decode('utf-8'))

okt = Okt()
texts = []
sentences = []

for item in data['items']:
    for field in ['title', 'description']:
        # HTML 태그, 특수문자 제거
        text = re.sub('<.*?>', '', item[field])
        texts.append(text)
        clean = re.sub('[^가-힣\s]', ' ', text)
        nouns = okt.nouns(clean)
        if nouns:
            sentences.append(nouns)

# 2. 명사 추출
all_nouns = sum(sentences, [])
print(all_nouns[:20])

# 3. 빈도분석
nouns_series = pd.Series(all_nouns)
freq = nouns_series.value_counts()
print(freq.head(20))

# 4. 워드클라우드
word_freq = freq.to_dict()
wordcloud = WordCloud(font_path='./data/NanumPenScript-Regular.ttf', background_color='white', width=800, height=400).generate_from_frequencies(word_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

# nltk.Text plot
text_obj = Text(all_nouns)
plt.figure(figsize=(10, 5))

if platform.system() == 'Darwin':
    plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux':
    plt.rc('font', family='Malgun Gothic') 

text_obj.plot(20)

# 5. Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, sg=1)
print(model.wv.most_similar('전주', topn=10))


TypeError: expected string or bytes-like object